# Licenses Dataset Exploration

We load a CSV of software licenses and make two simple Altair charts. One is a bar chart of counts by license. The second adds interactivity (hover selection) to highlight a specific license and show its proportion.

In [1]:
import pandas as pd
import altair as alt

# Allow large datasets
alt.data_transformers.disable_max_rows()

url = 'https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv'
df = pd.read_csv(url)
df.head()

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,NaN,NaN,NaN,N,08/16/2006,NaN,NaN,NaN,NaN,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,NaN,NaN,NaN,N,11/12/2021,NaN,NaN,NaN,NaN,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN


In [2]:
# Inspect column names
list(df.columns)

['_id',
 'License Type',
 'Description',
 'License Number',
 'License Status',
 'Business',
 'Title',
 'First Name',
 'Middle',
 'Last Name',
 'Prefix',
 'Suffix',
 'Business Name',
 'BusinessDBA',
 'Original Issue Date',
 'Effective Date',
 'Expiration Date',
 'City',
 'State',
 'Zip',
 'County',
 'Specialty/Qualifier',
 'Controlled Substance Schedule',
 'Delegated Controlled Substance Schedule',
 'Ever Disciplined',
 'LastModifiedDate',
 'Case Number',
 'Action',
 'Discipline Start Date',
 'Discipline End Date',
 'Discipline Reason']

In [3]:
# Basic aggregation: count rows per license
counts = (
    df['License Type']
    .value_counts()
    .rename_axis('license')
    .reset_index(name='count')
)
# Ensure numeric count just in case
import pandas as _pd
counts['count'] = _pd.to_numeric(counts['count'], errors='coerce').fillna(0).astype(int)
counts

,license,count
0,DETECTIVE BOARD,4867
1,COSMO,3781
2,DENTAL,739
3,FUNERAL AND EMBALMER,98
4,DIETETIC AND NUTRITION,73
5,DESIGN FIRM,71
6,MASSAGE LICENSING BD,52
7,HOME INSPECTOR,46
8,COMM ASSOC MGR,37
9,CLIN PSYCHOLOGIST,24


In [4]:
# Plot 1: Simple bar chart of license counts
bar = alt.Chart(counts).mark_bar(color='steelblue').encode(
    x=alt.X('license:N', sort='-y', title='License'),
    y=alt.Y('count:Q', title='Count')
)
bar

alt.Chart(...)

In [5]:
# Plot 2: Interactive - highlight on hover and show percentage label
selection = alt.selection_point(fields=['license'], on='mouseover')
counts['percent'] = (counts['count'] / counts['count'].sum() * 100).round(1)
interactive_bar = alt.Chart(counts).mark_bar().encode(
    x=alt.X('license:N', sort='-y', title='License'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.condition(selection, alt.value('orange'), alt.value('lightgray'))
).add_params(selection)
text = alt.Chart(counts).mark_text(dy=-6, color='black').encode(
    x=alt.X('license:N', sort='-y'),
    y='count:Q',
    text=alt.condition(selection, 'percent:Q', alt.value(''))
)
interactive = (interactive_bar + text)
interactive

alt.LayerChart(...)

In [6]:
# Save charts as vega-lite v5 JSON for embedding (robust writer)
import json, os, re
outdir = os.path.abspath(os.path.join('..', 'assets', 'json'))
os.makedirs(outdir, exist_ok=True)

def save_chart_v5(chart, path, min_bytes=500):
    """
    Robust save:
    1. Use chart.to_json() (Altair's stable serializer) to capture datasets.
    2. Force $schema to v5 (in case environment default is v6+).
    3. Write text; verify file size; if unexpectedly tiny fall back to to_dict()+json.dump.
    4. Raise if still too small so failure is visible.
    """
    # First pass via to_json (tends to be reliable vs json.dump on dict here)
    js = chart.to_json()
    js = re.sub(r'https://vega.github.io/schema/vega-lite/v\d+[^"\\]*',
                'https://vega.github.io/schema/vega-lite/v5.json', js, count=1)
    with open(path, 'w') as f:
        f.write(js)
    size1 = os.path.getsize(path)
    if size1 < min_bytes:
        # Fallback path: dict serialization
        spec = chart.to_dict()
        spec['$schema'] = 'https://vega.github.io/schema/vega-lite/v5.json'
        with open(path, 'w') as f:
            json.dump(spec, f)
        size2 = os.path.getsize(path)
        if size2 < min_bytes:
            raise RuntimeError(f"Spec write failed for {path}: size {size2} bytes < {min_bytes}")

# Export initial simple charts
save_chart_v5(bar, os.path.join(outdir, 'licenses_plot1.json'))
save_chart_v5(interactive, os.path.join(outdir, 'licenses_plot2.json'))
'exported-basic-v5'

'exported-basic-v5'

In [7]:
# Linked crossfilter: bar (License Type) + heatmap (License Status x License Type)
type_select = alt.selection_point(fields=['License Type'], on='click', toggle=True)

topN = 15
top_counts = counts.sort_values('count', ascending=False).head(topN)
top_counts_labeled = top_counts.rename(columns={'license':'License Type'})

left_bar = alt.Chart(top_counts_labeled).mark_bar().encode(
    y=alt.Y('License Type:N', sort='-x', title='License Type'),
    x=alt.X('count:Q', title='Total Count'),
    color=alt.condition(type_select, alt.value('orange'), alt.value('lightgray')),
    tooltip=['License Type:N','count:Q']
).add_params(type_select).properties(width=300, height=400)

status_df = (
    df[['License Type','License Status']]
    .dropna()
    .groupby(['License Type','License Status']).size()
    .reset_index(name='n')
)
status_top = status_df[status_df['License Type'].isin(top_counts_labeled['License Type'])]

heat = alt.Chart(status_top).mark_rect().encode(
    y=alt.Y('License Type:N', sort=top_counts_labeled['License Type'].tolist(), title=''),
    x=alt.X('License Status:N', title='Status'),
    color=alt.Color('n:Q', title='Count', scale=alt.Scale(scheme='blues')),
    opacity=alt.condition(type_select, alt.value(1.0), alt.value(0.5)),
    tooltip=['License Type:N','License Status:N','n:Q']
).properties(width=400, height=1000)

# Compose dashboard
dashboard = alt.hconcat(left_bar, heat).resolve_scale(color='independent')
dashboard

alt.HConcatChart(...)

In [ ]:
# Focus + Context Zoomable Time-Series (top 8 license types)
# Builds timeseries_chart used later for export.
import pandas as _pd, numpy as _np
# Ensure original DF present
if 'df' not in globals():
    url = 'https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv'
    df = _pd.read_csv(url)
# Parse date column robustly
date_col = None
for c in ['Original Issue Date','Issue Date','IssueDate','OriginalIssueDate']:
    if c in df.columns: date_col = c; break
if date_col is None: raise ValueError('No recognizable issue date column found.')
df['_IssueDT'] = _pd.to_datetime(df[date_col], errors='coerce')
df['IssueMonth'] = df['_IssueDT'].dt.to_period('M').dt.to_timestamp()
month_counts = (df.dropna(subset=['IssueMonth','License Type'])
                 .groupby(['IssueMonth','License Type']).size()
                 .reset_index(name='count'))
# Limit to top 8 overall license types for clarity
top8 = month_counts.groupby('License Type')['count'].sum().sort_values(ascending=False).head(8).index.tolist()
month_top = month_counts[month_counts['License Type'].isin(top8)]
# Simplify legend selection to avoid duplicate internal signal generation with layered marks.
legend_sel = alt.selection_point(fields=['License Type'], bind='legend')
# Brush for focus+context zoom
brush = alt.selection_interval(encodings=['x'])
base_ts = alt.Chart(month_top).encode(
    x=alt.X('IssueMonth:T', title='Issue Month'),
    y=alt.Y('count:Q', title='Monthly Count'),
    color=alt.Color('License Type:N', title='License Type'),
    tooltip=[
        alt.Tooltip('IssueMonth:T', title='Issue Month'),
        alt.Tooltip('License Type:N', title='License Type'),
        alt.Tooltip('count:Q', title='Count')
    ]
)
# Remove point layering (point=True) to prevent duplicated selection signals in layered marks.
detail = base_ts.mark_line().add_params(legend_sel).transform_filter(brush).properties(height=360, width='container')
overview = base_ts.mark_area(opacity=0.25).add_params(brush).properties(height=60, width='container')
# Compose
timeseries_chart = alt.vconcat(detail, overview).resolve_scale(color='independent')
timeseries_chart

alt.VConcatChart(...)

In [9]:
# Export advanced charts as vega-lite v5 JSON for embedding (timeseries + static dashboard)
adv_outdir = outdir  # reuse assets/json directory

# Save specs (enforce v5 schema & robustness)
save_chart_v5(timeseries_chart, os.path.join(adv_outdir, 'licenses_timeseries.json'))
save_chart_v5(dashboard, os.path.join(adv_outdir, 'licenses_dashboard.json'))
'exported-advanced-v5'

'exported-advanced-v5'

In [10]:
# Dynamic Top-N dashboard: external HTML controls drive these parameters (remove internal widgets)
# Build status table with counts and percentages
status_raw = df[['License Type','License Status']].dropna()
status_counts = (
    status_raw
    .groupby(['License Type','License Status']).size()
    .reset_index(name='n')
)
license_totals = status_counts.groupby('License Type')['n'].sum().reset_index(name='total')
full_status = status_counts.merge(license_totals, on='License Type')
full_status['percent'] = (full_status['n'] / full_status['total'] * 100).round(1)

# Ranked license counts for TopN filtering
counts_ranked = counts.copy()
counts_ranked['rank'] = counts_ranked['count'].rank(method='first', ascending=False).astype(int)

# Parameters (no bindings here — external controls will update these signals)
TopN = alt.param(value=20, name='TopN')  # default now 20, range enforced in HTML slider
Metric = alt.param(value='Count', name='Metric')

# Visual row step size so total height scales with number of license types
ROW_STEP = 24  # pixels per category row

# Main bar (TopN filtered)
bar_src = alt.Chart(counts_ranked).transform_filter('datum.rank <= TopN')
bar_dynamic = bar_src.mark_bar().encode(
    y=alt.Y('license:N', sort=alt.SortField(field='rank', order='ascending'), title='License'),
    x=alt.X('count:Q', title='Total Count'),
    tooltip=['license:N','count:Q']
).properties(width=420, height={'step': ROW_STEP})

# Heatmap (TopN filtered via lookup to get rank)
heat_src = alt.Chart(full_status).transform_lookup(
    lookup='License Type',
    from_=alt.LookupData(counts_ranked.rename(columns={'license':'License Type'}), 'License Type', ['count','rank'])
).transform_filter('datum.rank <= TopN')
heat_calc = heat_src.transform_calculate(value="Metric == 'Count' ? datum.n : datum.percent")
heat_dynamic = heat_calc.mark_rect().encode(
    y=alt.Y('License Type:N', sort=alt.SortField(field='rank', order='ascending'), title=''),
    x=alt.X('License Status:N', title='Status'),
    color=alt.Color('value:Q', title='Metric Value', scale=alt.Scale(scheme='teals')),
    tooltip=['License Type:N','License Status:N','n:Q','percent:Q','value:Q']
).properties(width=420, height={'step': ROW_STEP})

# Compose dashboard (no internal control layer)
main_dash = alt.hconcat(bar_dynamic, heat_dynamic).resolve_scale(color='independent').add_params(TopN, Metric)

# Export dynamic dashboard JSON without internal widget bindings (robust save)
save_chart_v5(main_dash, os.path.join(outdir, 'licenses_dashboard_plus.json'))
'saved-dynamic-v5'

'saved-dynamic-v5'

In [11]:
# Validation: report sizes of exported JSON specs
import os, json
spec_dir = os.path.abspath(os.path.join('..','assets','json'))
files = [
    'licenses_plot1.json',
    'licenses_plot2.json',
    'licenses_timeseries.json',
    'licenses_dashboard.json',
    'licenses_dashboard_plus.json'
]
report = {}
for fname in files:
    path = os.path.join(spec_dir, fname)
    if os.path.exists(path):
        report[fname] = os.path.getsize(path)
    else:
        report[fname] = 'MISSING'
print('Exported spec sizes (bytes):')
for k,v in report.items():
    print(f'  {k}: {v}')
# Quick sanity: open timeseries and show top-level keys
with open(os.path.join(spec_dir,'licenses_timeseries.json')) as f:
    import itertools
    head = ''.join(list(itertools.islice(f, 5)))
print('\nFirst lines of timeseries spec:\n', head)

Exported spec sizes (bytes):
  licenses_plot1.json: 3355
  licenses_plot2.json: 4428
  licenses_timeseries.json: 221311
  licenses_dashboard.json: 13199
  licenses_dashboard_plus.json: 26757

First lines of timeseries spec:
 {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "config": {
    "view": {
      "continuousHeight": 300,

